# Agent CFO — Performance Optimization & Design

---
This is the starter notebook for your project. Follow the required structure below.


You will design and optimize an Agent CFO assistant for a listed company. The assistant should answer finance/operations questions using RAG (Retrieval-Augmented Generation) + agentic reasoning, with response time (latency) as the primary metric.

Your system must:
*   Ingest the company’s public filings.
*   Retrieve relevant passages efficiently.
*   Compute ratios/trends via tool calls (calculator, table parsing).
*   Produce answers with valid citations to the correct page/table.


## 1. Config & Secrets

Fill in your API keys in secrets. **Do not hardcode keys** in cells.

In [ ]:
import os

# Example:
# os.environ['GEMINI_API_KEY'] = 'your-key-here'
# os.environ['OPENAI_API_KEY'] = 'your-key-here'

COMPANY_NAME = "DBS Bank"


## 2. Data Download (Dropbox)

*   Annual Reports: last 3–5 years.
*   Quarterly Results Packs & MD&A (Management Discussion & Analysis).
*   Investor Presentations and Press Releases.
*   These files must be submitted later as a deliverable in the Dropbox data pack.
*   Upload them under `/content/data/`.

Scope limit: each team will ingest minimally 15 PDF files total.


## 3. System Requirements

**Retrieval & RAG**
*   Use a vector index (e.g., FAISS, LlamaIndex) + a keyword filter (BM25/ElasticSearch).
*   Citations must include: report name, year, page number, section/table.

**Agentic Reasoning**
*   Support at least 3 tool types: calculator, table extraction, multi-document compare.
*   Reasoning must follow a plan-then-act pattern (not a single unstructured call).

**Instrumentation**
*   Log timings for: T_ingest, T_retrieve, T_rerank, T_reason, T_generate, T_total.
*   Log: tokens used, cache hits, tools invoked.
*   Record p50/p95 latencies.

In [ ]:
# TODO: Implement ingestion pipeline
import os
import time
import pickle
import logging
import pandas as pd
import numpy as np
import json
import re
from typing import List, Dict, Any, Tuple
from pathlib import Path

# RAG related libararies
import faiss
from sentence_transformers import SentenceTransformer
import fitz  # PyMuPDF for PDF processing
from rank_bm25 import BM25Okapi
import google.generativeai as genai # Gemini API for higher token limits

# Initialise logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# Tool for financial calculations
class CalculatorTool:
    def calculate_ratio(self, numerator: float, denominator: float, ratio_name: str = "") -> Dict[str, Any]:
        try:
            if denominator == 0:
                return {"error": f"Cannot calculate {ratio_name}: denominator is zero"}

            ratio = (numerator / denominator) * 100 if "ratio" in ratio_name.lower() else (numerator / denominator)
            return {
                "ratio_name": ratio_name,
                "numerator": numerator,
                "denominator": denominator,
                "result": round(ratio, 2),
                "formula": f"{numerator} / {denominator}"
            }
        except Exception as e:
            return {"error": str(e)}
        
    def trend_analysis(self, values: List[float], periods: List[str]) -> Dict[str, Any]:
        if len(values) != len(periods):
            return {"error": "Values and periods must have the same length"}
        
        if len(values) < 2:
            return {"error": "Need at least two data points for trend analysis"}
        
        # Calculate period-over-period changes
        changes = []
        for i in range(1, len(values)):
            if values[i-1] != 0:
                pct_change = ((values[i] - values[i-1]) / values[i-1]) * 100
                changes.append(round(pct_change, 2))
            else:
                changes.append(0)

        return {
            "periods": periods,
            "values": values,
            "period_changes": changes,
            "overall_trend": "increasing" if values[-1] > values[0] else "decreasing",
            "average_change": round(sum(changes) / len(changes), 2) if changes else 0
        }
    

# Tool for extracting table from dataset
class TableExtractionTool:
    def extract_financial_numbers(self, text: str) -> List[Dict[str, Any]]:
        # Pattern for numbers with currency/percentage
        patterns = [
            r'(\$|S\$|USD|SGD)?\s*(\d{1,3}(?:,\d{3})*(?:\.\d+)?)\s*(million|billion|thousand|m|bn|k)?',
            r'(\d{1,3}(?:,\d{3})*(?:\.\d+)?)\s*(%|percent|basis points|bps)',
            r'(NIM|CTI|ROE|ROA|CET1)\s*[:=]?\s*(\d+(?:\.\d+)?)\s*(%|bps)?'
        ]

        extracted = []
        for pattern in patterns:
            matches = re.finditer(pattern, text, re.IGNORECASE)
            for match in matches:
                extracted.append({
                    "text": match.group(0),
                    "value": match.group(2) if len(match.groups()) > 1 else match.group(1),
                    "context": text[max(0, match.start()-50):match.end()+50]  # 50 chars before and after
                })

        return extracted
    
    def parse_table_structure(self, text: str) -> Dict[str, Any]:
        lines = text.split('\n')
        table_lines = []

        for line in lines:
            # Look for lines that might be table rows (have multiple numbers/columns)
            if re.search(r'\d.*\d', line) and ('|' in line or '\t' in line or len(re.findall(r'\d+', line)) > 1):
                table_lines.append(line.strip())

        return {
            "potential_table_rows": table_lines[:10], # Return first 10 rows
            "row_count": len(table_lines)
        }
    

# Tool for comparing info across docs
class DocumentComparisonTool:
    def compare_metrics_across_docs(self, documents: List[Dict], metric_name: str) -> Dict[str, Any]:
        comparisons = []
        for doc in documents:
            # Extract metric from document text
            numbers = re.findall(r'\d+(?:\.\d+)?', doc.get('text', ''))
            filename = doc.get('metadata', {}).get('filename', 'unknown')
            
            comparisons.append({
                "document": filename,
                "metric_candidates": numbers[:5], # Return first 5 found numbers
                "text_snippet": doc.get('text', '')[:200] # First 200 chars
            })

        return {
            "metric_name": metric_name,
            "comparisons": comparisons
        }
        
# RAG functions
class CFORAGPipeline:
    def __init__(self, persist_dir="./cfo_rag_data"):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.persist_dir = persist_dir
        self.documents = []
        self.document_metadata = []
        self.index = None
        self.bm25 = None

        # Initialise tools
        self.calculator_tool = CalculatorTool()
        self.table_extraction_tool = TableExtractionTool()
        self.doc_comparison_tool = DocumentComparisonTool()

        # Create directory for persistence
        os.makedirs(self.persist_dir, exist_ok=True)

        # Performance tracking
        self.metrics = {
            'T_ingest': 0,
            'T_retrieve': 0,
            'T_rerank': 0,        
            'documents_ingested': 0,
            }
        
        logger.info("Initialized CFO RAG Pipeline")

    def extract_text_from_pdf(self, pdf_path: str) -> List[Dict[str, Any]]:
        # for document chunking
        chunks = []

        try:
            doc = fitz.open(pdf_path)
            filename = Path(pdf_path).stem

            for page_num in range(len(doc)):
                page = doc[page_num]
                text = page.get_text()

                if text.strip():
                    # split by sentences/paragraphs for chunking
                    paragraphs = text.split('\n\n')

                    for i, paragraph in enumerate(paragraphs):
                        if len(paragraph.strip()) > 50:
                            chunk = {
                                'text': paragraph.strip(),
                                'metadata': {
                                    'filename': filename,
                                    'page': page_num + 1,
                                    'chunk_id': f"{filename}_p{page_num+1}_c{i+1}",
                                    'source_type': self._classify_document_type(filename)
                                }
                            }
                            chunks.append(chunk)

            doc.close()
            logger.info(f"Extracted {len(chunks)} text chunks from {pdf_path}")

        except Exception as e:
            logger.error(f"Error extracting text from {pdf_path}: {e}")

        return chunks
    
    def _classify_document_type(self, filename: str) -> str:
        # based on filename
        filename_lower = filename.lower()
        if 'annual' in filename_lower:
            return 'annual_report'
        elif any(q in filename_lower for q in ['1q', '2q', '3q', '4q', 'quarter']):
            return 'quarterly_report'
        elif 'performance' in filename_lower:
            return 'performance_summary'
        else:
            return 'financial_report'
        
        # document ingestion from data directory containing PDFs/datasets
    def ingest_documents(self, data_dir: str = "./content/data") -> Dict[str, Any]:
        # record time taken to ingest the documents
        start_time = time.time()

        pdf_files = list(Path(data_dir).glob("*.pdf"))
        if not pdf_files:
            raise ValueError(f"No PDF files found in {data_dir}")
        
        all_chunks = []

        # process each PDF file
        for pdf_file in pdf_files:
            chunks = self.extract_text_from_pdf(str(pdf_file))
            all_chunks.extend(chunks)

        # separate text and metadata
        texts = [chunk['text'] for chunk in all_chunks]
        metadatas = [chunk['metadata'] for chunk in all_chunks]

        self.documents = texts
        self.document_metadata = metadatas

        # Create embeddings
        embeddings = self.model.encode(texts, convert_to_numpy=True, show_progress_bar=True)

        # Create FAISS index
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(embeddings.astype('float32'))

        # create BM25 index for keyword search
        tokenised_docs = [doc.lower().split() for doc in texts]
        self.bm25 = BM25Okapi(tokenised_docs)

        # save data
        self._save_data()

        # update metrics
        self.metrics['T_ingest'] = time.time() - start_time
        self.metrics['documents_ingested'] = len(texts)
        logger.info(f"Ingested {len(texts)} documents in {self.metrics['T_ingest']:.2f} seconds")

        return {
            'documents_processed': len(pdf_files),
            'chunks_created': len(texts),
            'ingestion_duration': self.metrics['T_ingest']
        }
    
    # retrieve relevant documents using hybrid search
    def hybrid_retrieve(self, query: str, top_k: int = 5) -> List[Dict[str, Any]]:
        start_time = time.time()

        if not self.documents or self.index is None:
            return []
        
        try: 
            # Vector search
            query_embedding = self.model.encode([query], convert_to_numpy=True)
            vector_k = min(top_k * 2, len(self.documents))
            distances, indices = self.index.search(query_embedding.astype('float32'), vector_k)

            # BM25 keyword search
            bm25_scores = self.bm25.get_scores(query.lower().split())

            # merge and rerank
            combined_results = []
            for i, idx in enumerate(indices[0]):
                if idx < len(self.documents):
                    # normalise scores and combine
                    vector_score = 1 / (1 + distances[0][i])  # convert distance to similarity
                    bm25_score = bm25_scores[idx] if idx < len(bm25_scores) else 0

                    combined_score = vector_score + bm25_score

                    result = {
                        'text': self.documents[idx],
                        'metadata': self.document_metadata[idx],
                        'combined_score': combined_score,
                        'vector_score': vector_score,
                        'bm25_score': bm25_score,
                        'citation': f"{self.document_metadata[idx]['filename']}, Page {self.document_metadata[idx]['page']}"
                    }
                    combined_results.append(result)

            # sort by combined score and take top k
            combined_results.sort(key=lambda x: x['combined_score'], reverse=True)
            final_results = combined_results[:top_k]

            self.metrics['T_retrieve'] = time.time() - start_time
            return final_results
        
        except Exception as e:
            logger.error(f"Error during retrieval: {e}")
            return []
        
    def _save_data(self):
        # Save FAISS index
        try:
            with open(os.path.join(self.persist_dir, 'documents.pkl'), 'wb') as f:
                pickle.dump(self.documents, f)

            with open(os.path.join(self.persist_dir, 'metadata.pkl'), 'wb') as f:
                pickle.dump(self.document_metadata, f)

            if self.index is not None:
                faiss.write_index(self.index, os.path.join(self.persist_dir, 'faiss_index.bin'))

            if self.bm25 is not None:
                with open(os.path.join(self.persist_dir, 'bm25.pkl'), 'wb') as f:
                    pickle.dump(self.bm25, f)
        
            logger.info("Saved ingestion data to disk")

        except Exception as e:
            logger.error(f"Error saving data: {e}")

cfo_rag = CFORAGPipeline()

# Ingest documents from data directory
print("=== Starting document ingestion ===")
ingestion_result = cfo_rag.ingest_documents(data_dir="./content/data")
print(f"Processed: {ingestion_result['documents_processed']} PDFs")
print(f"Created: {ingestion_result['chunks_created']} text chunks")
print(f"Ingestion Time: {ingestion_result['ingestion_duration']:.2f} seconds")

# Test retrieval
test_query = "Net Interest Margin trend over the past 3 years"
retrieved_docs = cfo_rag.hybrid_retrieve(test_query, top_k=3)

print(f"\n=== Retrieval Test ===")
print(f"Query: {test_query}")
print(f"Retrieved {len(retrieved_docs)} documents:")


if retrieved_docs:
    for i, doc in enumerate(retrieved_docs, 1):
        print(f"\nDocument {i}: {doc['citation']}")
        print(f"Combined Score: {doc['combined_score']:.4f}")
        print(f"Text Preview: {doc['text'][:150].replace(chr(10), ' ')}...")  # Print first 150 chars
else:
    print("No documents retrieved.")


## 4. Baseline Pipeline

**Baseline (starting point)**
*   Naive chunking.
*   Single-pass vector search.
*   One LLM call, no caching.

In [1]:
# TODO: Implement baseline retrieval + generation


## 5. Benchmark Runner

Run these 3 standardized queries. Produce JSON then prose answers with citations. These are the standardized queries.

*   Net Interest Margin (NIM) trend over last 5 quarters, values and 1–2 lines of explanation.
    *   Expected: quarterly financial highlights.
*   Operating Expenses (Opex) YoY for last 3 years; top 3 drivers from MD&A.
    *   Expected: Opex table + MD&A commentary.
*   Cost-to-Income Ratio (CTI) for last 3 years; show working + implications.
    *   Expected: Operating Income & Opex lines.


In [ ]:
# TODO: Implement benchmark runner


## 6. Instrumentation

Log timings: T_ingest, T_retrieve, T_rerank, T_reason, T_generate, T_total. Log tokens, cache hits, tools.

In [ ]:
# Example instrumentation schema
import pandas as pd
logs = pd.DataFrame(columns=['Query','T_ingest','T_retrieve','T_rerank','T_reason','T_generate','T_total','Tokens','CacheHits','Tools'])
logs

## 7. Optimizations

**Required Optimizations**

Each team must implement at least:
*   2 retrieval optimizations (e.g., hybrid BM25+vector, smaller embeddings, dynamic k).
*   1 caching optimization (query cache or ratio cache).
*   1 agentic optimization (plan pruning, parallel sub-queries).
*   1 system optimization (async I/O, batch embedding, memory-mapped vectors).

In [ ]:
# TODO: Implement optimizations


## 8. Results & Plots

Show baseline vs optimized. Include latency plots (p50/p95) and accuracy tables.

In [ ]:
# TODO: Generate plots with matplotlib


testing